POR HACER
* train validation test con data augmentation
* funcion data augmentation y guardar las fotos creadas

* cargar archivos txt, pickle o hd5py


HECHOS
* crear modelo cambiando la ultima capa y congelar capas a partir de la ultima convolucional
* cargar pesos
* compilar

In [ ]:
%run '../system/functions.ipynb' # import util functions

In [ ]:
from time import time # execution time no the best accurate but the easiest sometimes does not work

In [ ]:
# img_labels & img_list - completed path to file
PATH_TO_LABELS_FILE = "../data/output_dataset/img_labels"
PATH_TO_PATHS_FILE = "../data/output_dataset/img_list"

# output convnet files path + feed
PATH_TO_OUTPUT = "../data/output_convnet"

In [ ]:
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [4]:
n_categories = 8 # numero de categorias en las que clasificar las imagenes

In [1]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

Using TensorFlow backend.


In [2]:
# comprobar si el modelo es vgg16 como el implementado en keras
# https://github.com/keras-team/keras/blob/master/keras/applications/vgg16.py # estructura implementada en keras
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())

#top layer of the VGG net
model.add(Dense(4096, activation='relu',name="fc1"))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu',name="fc2"))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax',name="last_layer"))

# model.summary() # lo oculto por ahora

In [3]:
model.load_weights("keras_weights/vgg16_weights_tf_dim_ordering_tf_kernels.h5")

In [7]:
model.layers.pop()
model.outputs = [model.layers[-1].output]
model.layers[-1].outbound_nodes = []

# activacion softmax para calcular multiclasificacion
model.add(Dense(n_categories, activation='softmax',name="new_cats"))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
__________

In [ ]:
# for i, layer in enumerate(model.layers):
#     print (i, layer.name, layer.output_shape)

In [13]:
# congelamos todas salvo las fc1 fc2 y new_cats, en estas los pesos se actualizaran (entrenaran)
for layer in model.layers[:31]:
    layer.trainable = False

In [15]:
for i, layer in enumerate(model.layers):
    if layer.trainable:
        print("layer {0:s} is trainable".format(layer.name))
    else:
        print("layer {0:s} is freezed".format(layer.name))

layer zero_padding2d_1 is freezed
layer conv2d_1 is freezed
layer zero_padding2d_2 is freezed
layer conv2d_2 is freezed
layer max_pooling2d_1 is freezed
layer zero_padding2d_3 is freezed
layer conv2d_3 is freezed
layer zero_padding2d_4 is freezed
layer conv2d_4 is freezed
layer max_pooling2d_2 is freezed
layer zero_padding2d_5 is freezed
layer conv2d_5 is freezed
layer zero_padding2d_6 is freezed
layer conv2d_6 is freezed
layer zero_padding2d_7 is freezed
layer conv2d_7 is freezed
layer max_pooling2d_3 is freezed
layer zero_padding2d_8 is freezed
layer conv2d_8 is freezed
layer zero_padding2d_9 is freezed
layer conv2d_9 is freezed
layer zero_padding2d_10 is freezed
layer conv2d_10 is freezed
layer max_pooling2d_4 is freezed
layer zero_padding2d_11 is freezed
layer conv2d_11 is freezed
layer zero_padding2d_12 is freezed
layer conv2d_12 is freezed
layer zero_padding2d_13 is freezed
layer conv2d_13 is freezed
layer max_pooling2d_5 is freezed
layer flatten_1 is trainable
layer fc1 is train

In [16]:
# revisar estos hiperparametros --> leer sobre ellos y tomar decisiones
# sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

# compilamos este como el caso anterior para poder compararlos
model.compile(optimizer=SGD(), loss='categorical_crossentropy')

In [ ]:
# ahora ya hay que pasarle lista de fotografias a entrenar